In [ ]:
import pickle
import numpy as np
import networkx as nx
import pandas as pd
import brainmaptools as brainmap
import matplotlib.pyplot as plt
import community
import brainx.modularity as mod
import operator
%matplotlib inline
import csv
import scipy as scipy
import scipy.io as io
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import math as math
import pylab as pl
import My_functions as myfunctions
import progressbar as pb
import time
from collections import Counter
import random
import os
%load_ext Cython
import bct
from numba import jit

In [ ]:
domains= ['Memory', 'Working Memory', 'Emotion', 'Attention', 'Language', 'Vision', 'Audition', 'Base', 'Rest']

In [ ]:
relabel_dict_file='/Users/owner/Functional_Connectivity/Scripts/Label_dict.pck'
relabel_dict=pickle.load(open(relabel_dict_file,"rb"))

In [ ]:
G_nothresh={}
for x in domains:
    filename='G_nothresh_jaccard_'+x+'_.pck'
    workdir='/Users/owner/Functional_Connectivity/Results/G_jaccard_graphs/Unthresholded_G_jaccard/'
    output=workdir+filename
    G_nothresh[x]=pickle.load(open (output, 'rb'))

# Random Null Graph Modularity

In [ ]:
%%cython
cpdef random_graphs(G, list domain_list):
    from __main__ import nx
    from __main__ import time
    cdef int iterations = 1000
    cdef int nodes= G.number_of_nodes()
    cdef int edges = G.number_of_edges()
    cdef int n
    cdef dict rand_graph_dict={}
    for n in range(iterations):
        rand_graph_dict[n]=nx.gnm_random_graph(nodes, edges)

    return rand_graph_dict

In [ ]:
for i in pb.progressbar(range(1000)):
    G_random={x: random_graphs(G_nothresh[x], domains) for x in domains}

# BCTPY Newman

In [ ]:
%%cython
cpdef bct_mod_dict(G, list domain_list):
    from __main__ import nx
    from __main__ import bct
    from __main__ import np
    cdef dict bct_rand={}
    cdef int n
    cdef str x
    bct_rand={x: {n: bct.modularity_und(np.squeeze(np.asarray(nx.to_numpy_matrix(G[x][n])))) for n in range(len(G[x]))} for x in domain_list}
    
    return bct_rand

In [ ]:
for i in pb.progressbar(range(1000)):
    bct_rand_mod=bct_mod_dict(G_random, domains)

In [ ]:
@jit
def bct_mod_list(mod_dict, domain_list):
    bct_list={}
    for x in domain_list:
        bct_list[x]=[mod_dict[x][n][1] for n in range(len(mod_dict[x]))]
    return bct_list

In [ ]:
bct_rand_mod_list=bct_mod_list(bct_rand_mod, domains)

In [ ]:
pickle.dump(bct_rand_mod_list, open('bct_rand_mod_list.pck', 'wb'))

# Louvain

In [ ]:
%%cython
from __main__ import community
from __main__ import nx
from __main__ import G_random
from __main__ import domains
cpdef lou_mod_dict(G, list domain_list):
    cdef str x
    cdef int n
    lou_rand={x: {n: community.best_partition(G[x][n]) for n in range(len(G[x]))} for x in domain_list}
    return lou_rand

In [ ]:
for i in pb.progressbar(range(1000)):
    rand_part=lou_mod_dict(G_random, domains)

In [ ]:
@jit
def lou_rand_list(G, lou_dict, domain_list):
    lou_list={}
    for x in domain_list:
        lou_list[x]=[community.modularity(lou_dict[x][n], G[x][n]) for n in range(len(G[x]))]
    return lou_list

In [ ]:
lou_rand_mod_list=lou_rand_list(G_random, rand_part, domains)

In [ ]:
pickle.dump(lou_rand_mod_list, open('lou_rand_mod_list.pck', 'wb'))

# Consensus

In [ ]:
%%cython
from __main__ import nx
from __main__ import brainmap
cpdef bct_rand_g_partition(G, dict bct_dict, list domain_list):
    cdef dict bct_rand_part
    cdef str x
    cdef int n
    bct_rand_part={x: {n: dict(zip(G[x][n].nodes(), bct_dict[x][n][0])) for n in range(len(G[x]))} for x in domain_list}
    bct_rand_partition={x: {n: brainmap.make_brainx_style_partition (bct_rand_part[x][n]) for n in range(len(bct_rand_part[x]))} for x in domain_list}
    return bct_rand_partition

In [ ]:
rand_bct_part=bct_rand_g_partition(G_random, bct_rand_mod, domains)

In [ ]:
%%cython
from __main__ import nx
from __main__ import np
cpdef consen_rand_array(G, lou_part, bct_part, list domain_list):
    cdef dict random_matrixs
    cdef dict sum_matrix
    cdef dict avg_rand_matrix
    cdef str x
    cdef int n
    cdef int r
    cdef int d
    cdef int k
    cdef int b
    cdef int v
    random_matrixs={x: {n: np.zeros([G[x][n].number_of_nodes(),G[x][n].number_of_nodes()]) for n in range(len(G[x]))} for x in domain_list}
    sum_matrix={x: {n: np.zeros([G[x][n].number_of_nodes(),G[x][n].number_of_nodes()]) for n in range(len(G[x]))} for x in domain_list}
    for x in domain_list:
        for n in range(len(G[x])):
            for r,k in enumerate (G[x][n].nodes()):
                for d,b in enumerate (G[x][n].nodes()):
                    if k != b:
                        if lou_part[x][n][k] == lou_part[x][n][b]:
                            random_matrixs[x][n][r][d] += 1
                        for v in range (len(bct_part[x][n])):
                            if k in bct_part[x][n][v]:
                                if b in bct_part[x][n][v]:
                                    random_matrixs[x][n][r][d] += 1
    sum_matrix={x: {n: random_matrixs[x][n]+sum_matrix[x][n] for n in range(len(G[x]))} for x in domain_list}
    avg_rand_matrix={x: {n: sum_matrix[x][n]/2 for n in range(len(G[x]))} for x in domain_list}
    return avg_rand_matrix

In [ ]:
for i in pb.progressbar(range(1000)):
    rand_consensus_array=consen_rand_array(G_random, rand_part, rand_bct_part, domains)

In [ ]:
%%cython
from __main__ import np
from __main__ import nx
cpdef random_consensus_G (G, dict rand_con_array, list domain_list):
    cdef str x
    cdef int n
    cdef dict n_nodes = {}
    cdef dict nodes = {}
    cdef dict con_G = {}
    for x in domain_list:
        n_nodes[x]={n: len(range(nx.number_of_nodes(G[x][n]))) for n in range(len(G[x]))}
        nodes[x]={n: dict(zip(range(n_nodes[x][n]), G[x][n].nodes())) for n in range(len(G[x]))}
        con_G[x]={n: nx.from_numpy_matrix(rand_con_array[x][n]) for n in range(len(G[x]))}
        con_G[x]={n: nx.relabel_nodes(con_G[x][n], nodes[x][n]) for n in range(len(G[x]))}

In [ ]:
random_con_G_dict=random_consensus_G(G_random, rand_consensus_array, domains)

In [ ]:
@jit
def com_rand_list(G, random_dict, domain_list):
    com_mod_list={}
    for x in domains:
        com_mod_list[x]=[community.modularity(random_dict[x][n], G[x][n]) for n in range(len(G[x]))]
    return com_mod_list

In [ ]:
rand_com_lou=com_rand_list(G_random, rand_consensus_array, domains)

In [ ]:
pickle.dump(rand_com_lou, open('rand_com_lou.pck', 'wb'))